In [5]:
from lxml import etree
import requests
import json
import pandas as pd
import re

In [2]:
def getDissid(sin, ein):
    url = 'https://c.y.qq.com/splcloud/fcgi-bin/fcg_get_diss_by_tag.fcg'
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
        'cookie': 'RK=7dNm4/X+Yj; tvfe_boss_uuid=bf00ee54e9081ab4; pgv_pvi=8772238336; pac_uid=1_857193777; pgv_pvid=6457341280; o_cookie=80; ptcz=c761e59c8c8d6bd5198866d02a5cb7313af1af468006c455d6c2b5d26201d42e; pgv_si=s10759168; _qpsvr_localtk=0.08285763449905015; ptisp=ctc; luin=o0857193777; lskey=00010000228dd1371b945c68ecfd3b71d3071425024a7a8a2a23e3ffcb5b9904c9f7088d2ea8c01539ffed92; pt2gguin=o0857193777; uin=o0857193777; skey=@Kydi7w0EI; p_uin=o0857193777; p_skey=HjsE9sEjznJfXk*9KFEeW4VZr6i3*tlXZ2nuzEw8kCg_; pt4_token=c-p6sv3JEboA51cSQ3ABqxM8O80Jct3jYYkgy-aEQuE_; p_luin=o0857193777; p_lskey=000400008f9c296cd10c03a5173d22a184aad124d791568e90e4198beb8ad699a4d02fbfc059f71ab3d8758c; ts_last=y.qq.com/portal/playlist.html; ts_refer=ui.ptlogin2.qq.com/cgi-bin/login; ts_uid=3392060960',
        'referer': 'https://y.qq.com/portal/playlist.html'
    }
    paramter = {
        'g_tk': '1089387893',
        'jsonpCallback': 'getPlaylist',
        'loginUin': '0',
        'hostUin': '0',
        'format': 'jsonp',
        'inCharset': 'utf8',
        'outCharset': 'utf-8',
        'notice': '0',
        'platform': 'yqq',
        'needNewCode': '0',
        'categoryId': '10000000',
        'sortId': '5',
        'sin': sin,  # 开始结点
        'ein': ein  # 结束结点，用于翻页
    }
    html = requests.get(url=url, params=paramter, headers=header)
    res = json.loads(html.text.lstrip('getPlaylist(').rstrip(')'))['data']['list']
    data = []
    if res != []:
        for t_item in res:
            item = {}
            ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')  # 用于去掉非法字符
            item['createtime'] = t_item['createtime']
            item['creator_qq'] = t_item['creator']['qq']
            item['creator_name'] = t_item['creator']['name']
            item['creator_name'] = ILLEGAL_CHARACTERS_RE.sub(r'', item['creator_name'])
            item['creator_isVip'] = t_item['creator']['isVip']
            item['dissid'] = t_item['dissid']  # 提取歌单id，用于后续提取歌曲id
            item['dissname'] = t_item['dissname']  # 歌单名称
            item['dissname'] = ILLEGAL_CHARACTERS_RE.sub(r'', item['dissname'])
            item['listennum'] = t_item['listennum']  # 播放量
            data.append(item)
    return data

In [17]:
# 爬取歌曲id
def getSongid(dissid):
    url = 'https://c.y.qq.com/qzone/fcg-bin/fcg_ucc_getcdinfo_byids_cp.fcg'
    header = {
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/59.0.3071.115 Safari/537.36',
        'cookie': 'RK=7dNm4/X+Yj; tvfe_boss_uuid=bf00ee54e9081ab4; pgv_pvi=8772238336; pac_uid=1_857193777; pgv_pvid=6457341280; o_cookie=857193777; ptcz=c761e59c8c8d6bd5198866d02a5cb7313af1af468006c455d6c2b5d26201d42e; pgv_si=s10759168; _qpsvr_localtk=0.08285763449905015; ptisp=ctc; luin=o0857193777; lskey=00010000228dd1371b945c68ecfd3b71d3071425024a7a8a2a23e3ffcb5b9904c9f7088d2ea8c01539ffed92; pt2gguin=o0857193777; uin=o0857193777; skey=@Kydi7w0EI; p_uin=o0857193777; p_skey=HjsE9sEjznJfXk*9KFEeW4VZr6i3*tlXZ2nuzEw8kCg_; pt4_token=c-p6sv3JEboA51cSQ3ABqxM8O80Jct3jYYkgy-aEQuE_; p_luin=o0857193777; p_lskey=000400008f9c296cd10c03a5173d22a184aad124d791568e90e4198beb8ad699a4d02fbfc059f71ab3d8758c; ts_last=y.qq.com/portal/playlist.html; ts_refer=ui.ptlogin2.qq.com/cgi-bin/login; ts_uid=3392060960',
        'referer': 'https://y.qq.com/n/yqq/playlist/{}.html'.format(dissid)
    }
    paramters = {
        'type': '1',
        'json': '1',
        'utf8': '1',
        'onlysong': '0',
        'disstid': dissid,
        'format': 'jsonp',
        'g_tk': '1089387893',
        'jsonpCallback': 'playlistinfoCallback',
        'loginUin': '857193777',
        'hostUin': '0',
        'inCharset': 'utf8',
        'outCharset': 'utf-8',
        'notice': 0,
        'platform': 'yqq',
        'needNewCode': 0
    }
    html = requests.get(url=url, params=paramters, headers=header)
    cdlist = json.loads(html.text.lstrip('playlistinfoCallback(').rstrip(')'))['cdlist']
    if len(cdlist) >= 1:
        cdlist = cdlist[0]
    data2 = []
    ILLEGAL_CHARACTERS_RE = re.compile(r'[\000-\010]|[\013-\014]|[\016-\037]')
    tags = ','.join([i['name'] for i in cdlist['tags']])
    tagsid = ','.join([str(i['id']) for i in cdlist['tags']])
    for i in cdlist['tags']:
        tagtmp = {}
        tagtmp['id'] = i['id']
        tagtmp['name'] = i['name']
        if tagtmp not in taglst:
            taglst.append(tagtmp)
    visitnum = cdlist['visitnum']
    for item in cdlist['songlist']:
        tmp = {}
        tmp['albumname'] = item['albumname']
        tmp['songname'] = item['songname']
        tmp['singer'] = ','.join([i['name'] for i in item['singer']])
        tmp['singermid'] = ','.join([i['mid'] for i in item['singer']])
        tmp['songmid'] = item['songmid']
        tmp['tags'] = tags
        tmp['tagsid'] = tagsid
        tmp['visitnum'] = visitnum
        tmp['albummid'] = item['albummid']
        tmp['url'] = 'https://y.qq.com/n/yqq/song/'+item['songmid']+'.html'
        data2.append(tmp)
        for i in item['singer']:
            singertmp = {}
            singertmp['mid'] = i['mid']
            singertmp['name'] = i['name']
            if singertmp not in singerlst:
                singerlst.append(singertmp)
        albumtmp = {}
        albumtmp['albummid'] = item['albummid']
        albumtmp['albumname'] = item['albumname']
        if albumtmp not in albumlst:
            albumlst.append(albumtmp)
    # return [data1,data2]
    return data2

In [62]:
sin = 0
ein = sin + 29
dissid_list = []
while True:
    data = getDissid(sin, ein)
    dissid_list.extend(data)
    sin = sin + 30
    ein = sin + 29
    if ein > 1030:
        break
t_data = pd.DataFrame(dissid_list)
t_data.to_csv('dissid.csv', index = False)
t_data

,createtime,creator_qq,creator_name,creator_isVip,dissid,dissname,listennum
0,2019-11-26,757633910,小飞侠.DAYTOY,0,885993631,肖战的歌 我们的歌,5415859
1,2020-03-08,1152921504819784901,Happy.,0,6949707670,下班来劲 | 来首电子，抛开烦恼,7608020
2,2019-05-24,3163854005,韩达达,2,7004110454,韩剧OST女王的催泪神作,1987444
3,2020-05-24,2374133080,LANNÉ,0,7564229425,小众兼宝藏·冷门又不失格调,1200120
4,2020-09-27,1847569460,.落差,0,7658655210,你那么温柔，一定很孤单吧,1769620
...,...,...,...,...,...,...,...
1015,2017-05-26,1605570521,Liquor.,0,1157503878,纯音乐钢琴曲:刷题必备 失眠良药,4468494
1016,2020-10-09,2562406741,Vuxa,0,7743173298,节奏BGM：鼓点遇上绝妙旋律,2220886
1017,2020-08-30,3077344505,精彩华辉,0,7698826019,「怀旧流行」都是青春的回忆啊,1731131
1018,2019-09-18,2298105529,我又可以了,2,7199053194,【华语精选】岁月沉淀的佳品,7974943


In [6]:
filename = 'dissid.csv'
names = ['createtime','creator_qq','creator_name','creator_isVip','dissid','dissname','listennum']
t_data = pd.read_csv(filename, names = names, encoding='utf-8')
t_data.drop([0], inplace = True)
t_data.reset_index(drop=True)

,createtime,creator_qq,creator_name,creator_isVip,dissid,dissname,listennum
0,2019-11-26,757633910,小飞侠.DAYTOY,0,885993631,肖战的歌 我们的歌,5415859
1,2020-03-08,1152921504819784901,Happy.,0,6949707670,下班来劲 | 来首电子，抛开烦恼,7608020
2,2019-05-24,3163854005,韩达达,2,7004110454,韩剧OST女王的催泪神作,1987444
3,2020-05-24,2374133080,LANNÉ,0,7564229425,小众兼宝藏·冷门又不失格调,1200120
4,2020-09-27,1847569460,.落差,0,7658655210,你那么温柔，一定很孤单吧,1769620
...,...,...,...,...,...,...,...
1014,2017-05-26,1605570521,Liquor.,0,1157503878,纯音乐钢琴曲:刷题必备 失眠良药,4468494
1015,2020-10-09,2562406741,Vuxa,0,7743173298,节奏BGM：鼓点遇上绝妙旋律,2220886
1016,2020-08-30,3077344505,精彩华辉,0,7698826019,「怀旧流行」都是青春的回忆啊,1731131
1017,2019-09-18,2298105529,我又可以了,2,7199053194,【华语精选】岁月沉淀的佳品,7974943


In [31]:
taglst = []
singerlst = []
songlst = []
albumlst = []
sing_list = []
dissids= t_data['dissid']
for i in dissids:
    print(i)
    d2 = getSongid(i)
    sing_list.extend(d2)
all_data = pd.DataFrame(sing_list)

885993631
6949707670
7004110454
7564229425
7658655210
7690002111
7229893667
7587920319
7711339742
7212954390
7708477791
7342928643
7382629476
5217738880
7765118504
1437721458
7123372894
7673177442
7735682764
7678657343
7712802404
1136199129
2356366127
7461790879
1788824062
3517857779
7567404842
7715821645
7520334743
7785460928
2391277948
7242168276
7778278548
7535283994
7808373086
5863290029
7498059409
7692240932
7537552719
7723138577
1469897849
7618560152
7710032313
7690336123
2931035154
7637214383
7685396575
7613169172
7532670538
7687815897
7060355343
7784481513
7808176854
7720311890
6746013591
7807886978
7691838312
7522943324
7283856320
7581343266
7683199209
4580773716
7798496732
1164657023
7084698351
7675764598
7810389838
1721400911
5375317625
7000491330
7798823985
1742108428
1452052493
7706788408
7727383695
7710032695
3540484302
2683955247
3658457678
7499337095
7733073935
7639593815
7779294041
7657332746
2919117025
7530172964
7613171044
7572797161
7666951595
3748979450
7767362229


6269155117
4208690011
7589000260
7377903318
3704623651
7720760237
2016907260
7442434959
7543465349
3856616098
7110372103
7695587512
2370564763
7583154525
7467921306
7682677139
7623920550
7539490149
7610357786
7653816755
2952171996
2679960417
7516141212
4237193954
7734900537
7687118062
1855128183
7614442512
7537787795
7603777938
2342856359
7467921369
7661064283
7619167305
7685440801
7720317576
872839954
7562151214
7606558023
7475319852
7097197259
7613372709
7671237418
7523077869
7109787308
1472776306
3573607299
3307888020
7576423412
7673314520
1719941120
7459588069
7559561831
2370564744
7174019146
2067204491
7635812681
7746001923
7020619438
3277068038
2067212997
7222867472
7739024426
7559412657
7716965708
7599877326
7668841692
7306068541
3559890884
1789072718
7739842973
7712614773
7624215376
7567376184
7563681381
2384755098
2349071298
2944217623
7599082024
7658515147
1167623944
7123892256
3108781778
7093152108
7424768225
7646494114
7521981320
7459592100
2370565162
7710119458
7504150326


PermissionError: [Errno 13] Permission denied: 'song.csv'

In [27]:
taglst = []
singerlst = []
songlst = []
albumlst = []
sing_list = []
d2 = getSongid('885993631')
sing_list.extend(d2)
all_data = pd.DataFrame(sing_list)
all_data

,albumname,songname,singer,singermid,songmid,tags,tagsid,visitnum,albummid,url
0,余生，请多指教 (2020星光大赏Live版),余生，请多指教 (2020星光大赏Live版),"杨紫,肖战","003XbbyD06iu1p,0022eAG537I1bg",002lsS4C1fxAzC,"国语,流行,现场音乐","165,6,197",5454276,003V1Vao3OS3sb,https://y.qq.com/n/yqq/song/002lsS4C1fxAzC.html
1,最幸运的幸运 (2020星光大赏Live版),最幸运的幸运 (2020星光大赏Live版),肖战,0022eAG537I1bg,002lNgBh2yBIHi,"国语,流行,现场音乐","165,6,197",5454276,002iScmt2I0LsP,https://y.qq.com/n/yqq/song/002lNgBh2yBIHi.html
2,,用尽我的一切奔向你（Live）,肖战,0022eAG537I1bg,000pEp703fs4zZ,"国语,流行,现场音乐","165,6,197",5454276,,https://y.qq.com/n/yqq/song/000pEp703fs4zZ.html
3,,夜空中最亮的星 (Live),肖战,0022eAG537I1bg,002MU4Hf28GIZO,"国语,流行,现场音乐","165,6,197",5454276,,https://y.qq.com/n/yqq/song/002MU4Hf28GIZO.html
4,陈情令 国风音乐专辑,【魏无羡】曲尽陈情,肖战,0022eAG537I1bg,001vl06k1uzyie,"国语,流行,现场音乐","165,6,197",5454276,002Tryw33HIGNP,https://y.qq.com/n/yqq/song/001vl06k1uzyie.html
...,...,...,...,...,...,...,...,...,...,...
60,《Keep Online》 NO版,你想我吗,"彭楚粤,伍嘉成,谷嘉诚,肖战","003k3wLI2BbZaV,003lLRuV2pqtct,004H64zR2nkx91,0...",001H2Lv723rAy6,"国语,流行,现场音乐","165,6,197",5454276,0009NbmJ1egERM,https://y.qq.com/n/yqq/song/001H2Lv723rAy6.html
61,《青春为祖国歌唱》第一辑经典篇,我和我的祖国,"李谷一,肖战,魏允熙,陈学冬","004Zi1th1ukmJV,0022eAG537I1bg,0008LEQv1ODodY,0...",002lQMOg1BMnIt,"国语,流行,现场音乐","165,6,197",5454276,002S9jO42HiqOT,https://y.qq.com/n/yqq/song/002lQMOg1BMnIt.html
62,,小星星 (Live),肖战,0022eAG537I1bg,000zoCpp4O9rgb,"国语,流行,现场音乐","165,6,197",5454276,001ZaCQY2OxVMg,https://y.qq.com/n/yqq/song/000zoCpp4O9rgb.html
63,,宝贝 (Live),肖战,0022eAG537I1bg,000kUxcQ0XAm3F,"国语,流行,现场音乐","165,6,197",5454276,001ZaCQY2OxVMg,https://y.qq.com/n/yqq/song/000kUxcQ0XAm3F.html


In [33]:
album = pd.DataFrame(albumlst)
album.to_csv('album.csv', index = False)

In [35]:
singer = pd.DataFrame(singerlst)
singer.to_csv('singer.csv', index = False)

In [37]:
tag = pd.DataFrame(taglst)
tag.to_csv('tag.csv', index = False)

In [39]:
all_data.to_csv('song.csv', index = False)